In [ ]:
import os
import sys
import glob
import shutil
import warnings
from datetime import datetime
from timeit import default_timer as timer

print("")
print("+"*shutil.get_terminal_size().columns)
print("Hierarchical Bayesian Analysis of Kepler Eccentricities")
print("Initialized {0}".format(datetime.now().strftime("%d-%b-%Y at %H:%M:%S")))
print("+"*shutil.get_terminal_size().columns)
print("")

# track date
YYYYMMDD = datetime.now().strftime("%Y%m%d")

# start program timer
global_start_time = timer()

## Parse arguments

In [ ]:
import argparse
import random
import string

try:
    # read the arguments
    parser = argparse.ArgumentParser(description="Inputs for Kepler hierarchical eccentricities analysis")
    parser.add_argument("--project_dir", default='/data/user/gjgilbert/projects/kepler-ecc-rp/', type=str, required=False, \
                        help="project directory for storing inputs and outputs")
    parser.add_argument("--data_dir", default='/data/user/gjgilbert/data/DR25_chains/', type=str, required=False, \
                        help="data directory for accessing DR25 posterior chains")
    parser.add_argument("--run_id", default=None, type=str, required=True, \
                        help="unique run identifier")
    parser.add_argument("--distribution", default='empirical', type=str, required=False, \
                        help="probability density function to use; can be 'histogram', 'empirical', 'beta', 'halfnormal', 'expon', 'rayleigh', 'invpareto'")
    parser.add_argument("--nsamp", default=1000, type=int, required=False, \
                        help="number of (re)samples to feed into hbayes model")
    parser.add_argument("--nbin", default=100, type=int, required=False, \
                        help="number of bins for probability density function'")

    parser.add_argument("--multiplicity", default=(1,99), nargs=2, type=int, required=False, \
                        help="(lower,upper) limits on multiplicity")
    parser.add_argument("--per_lim", default=(1,300), nargs=2, type=float, required=False, \
                        help="(lower,upper) limits on period")
    parser.add_argument("--rad_type", default=None, type=str, required=True, \
                        help="radius type to use' can by 'rp', 'rp10', or 'rpadj'")
    parser.add_argument("--rad_lim", default=None, nargs=2, type=float, required=True, \
                        help="(lower,upper) limits on radius; set lower=upper to use Gaussian binning")
    parser.add_argument("--rad_fwhm", default=None, type=float, required=False, \
                        help="fractional FWHM on radius bins if using Gaussian binning'")

    parser.add_argument("--mstar_lim", default=(0.,10.), nargs=2, type=float, required=False, \
                        help="(lower,upper) limits on stellar mass")
    parser.add_argument("--rstar_lim", default=(0.7,1.4), nargs=2, type=float, required=False, \
                        help="(lower,upper) limits on stellar radius")
    parser.add_argument("--feh_lim", default=(-0.6,0.6), nargs=2, type=float, required=False, \
                        help="(lower,upper) limits on stellar metallicity")
    parser.add_argument("--teff_lim", default=(4700,6500), nargs=2, type=float, required=False, \
                        help="(lower,upper) limits on stellar effective temperature")
    parser.add_argument("--age_lim", default=(0,14), nargs=2, type=float, required=False, \
                        help="(lower,upper) limits on stellar age")

    parser.add_argument("--e_detprior", default=1, type=int, required=False, \
                        help="flag to use geometric eccentricity detection prior (1) or not (0)")
    parser.add_argument("--b_detprior", default=0, type=int, required=False, \
                        help="flag to use impact parameter detection prior (1) or not (0)")

    # parse the arguments
    args = parser.parse_args()
    PROJECT_DIR  = args.project_dir
    DATA_DIR     = args.data_dir
    RUN_ID       = args.run_id
    DISTRIBUTION = args.distribution
    NSAMP        = args.nsamp
    NBIN         = args.nbin
    MULTIPLICITY = args.multiplicity
    PER_LIM      = args.per_lim
    RAD_TYPE     = args.rad_type
    RAD_LIM      = args.rad_lim
    RAD_FWHM     = args.rad_fwhm
    MSTAR_LIM    = args.mstar_lim
    RSTAR_LIM    = args.rstar_lim
    FEH_LIM      = args.feh_lim
    TEFF_LIM     = args.teff_lim
    AGE_LIM      = args.age_lim
    E_DETPRIOR   = bool(args.e_detprior)
    B_DETPRIOR   = bool(args.b_detprior)


except:
    PROJECT_DIR  = '/Users/research/projects/kepler-ecc-rp/'
    DATA_DIR     = '/Users/research/data/DR25_chains/'
    RUN_ID        = 'development_'
    DISTRIBUTION = 'empirical'
    NSAMP        = 1000
    NBIN         = 100
    MULTIPLICITY = (1,1)
    PER_LIM      = (1,300)
    RAD_TYPE     = 'rp10'
    #RAD_LIM      = (0.5,16)
    #RAD_LIM      = (1.36, 1.50)
    #RAD_LIM      = (1.50, 1.67)  # -10% super-Earths
    RAD_LIM      = (1.67, 2.02)  # +/- 10% fractional width centered on nominal radius valley
    #RAD_LIM      = (2.02, 2.23)  # +10% sub-Neptunes
    #RAD_LIM      = (2.23, 2.45)
    RAD_FWHM     = None          # fractional full width half max of Gaussian bins (if appliable)
    MSTAR_LIM    = (0.,10.)
    RSTAR_LIM    = (0.7,1.4)
    FEH_LIM      = (-0.6,0.6)
    TEFF_LIM     = (4700,6500)
    AGE_LIM      = (0,14)
    E_DETPRIOR   = True
    B_DETPRIOR   = False

    # auto-generate random run_id
    ascii = string.ascii_letters + string.digits
    for i in range(8):
        RUN_ID += random.choice(ascii)
    
    
RESULTS_DIR = os.path.join(PROJECT_DIR, 'Results', YYYYMMDD)
os.makedirs(RESULTS_DIR, exist_ok=True)

print(" Distribution: {0}".format(DISTRIBUTION))
print("")
print("   npl = {0}-{1}".format(MULTIPLICITY[0], MULTIPLICITY[1]))
print("   P   = {0}-{1}".format(PER_LIM[0], PER_LIM[1]))
print("   {0} = {1}-{2}".format(RAD_TYPE, RAD_LIM[0], RAD_LIM[1]))
print("")

## Import packages and define constants

In [ ]:
import astropy.constants as apc
from   copy import deepcopy
import diptest
import h5py
import matplotlib.pyplot as plt
import matplotlib.cm as cmap
from   matplotlib.colors import LinearSegmentedColormap
import numpy as np
import pandas as pd
from   scipy.interpolate import interp1d, UnivariateSpline, CubicSpline
from   scipy import stats
from   scipy.special import erf, erfinv
import seaborn as sns
import warnings

import aesara_theano_fallback.tensor as T
from   aesara_theano_fallback import aesara as theano
from   celerite2.theano import GaussianProcess
from   celerite2.theano import terms as GPterms
import pymc3 as pm
import pymc3_ext as pmx

sys.path.append(PROJECT_DIR)
from utils.astro import calc_T14_circ, calc_sma, calc_aRs, jacobian, detection_prior, duration_ratio
from utils.distributions import BetaDistLogPDF, NormDistLogPDF, ExponDistLogPDF, RayleighDistLogPDF, InvParetoDistLogPDF
from utils.eccsamp import imp_sample_rhostar
from utils.io import load_dr25_data_from_hdf5
from utils.models import build_simple_model, build_multilevel_model
from utils.stats import weighted_percentile, draw_random_samples

pi = np.pi

RSAU = (apc.R_sun/apc.au).value                                 # solar radius [AU]
RSRE = (apc.R_sun/apc.R_earth).value                            # R_sun/R_earth
RHOSUN_GCM3 = (3*apc.M_sun/(4*pi*apc.R_sun**3)).value/1000      # solar density [g/cm^3]

## Load DR25 data

In [ ]:
print("Loading data...")

#### Load catalog

In [ ]:
DR25_CATALOG = os.path.join(PROJECT_DIR, 'Catalogs/kepler_dr25_gaia_dr2_crossmatch.csv')
catalog = pd.read_csv(DR25_CATALOG, index_col=0)

# hard-code period and radius limits
use  = (catalog.period > 1) * (catalog.period < 300)
use *= (catalog.rp > 0) * (catalog.rp < 16)

# remove likely false positives
use *= catalog.fpp < 0.1

# require better than 20% precision on radius
use *= catalog.rp_err/catalog.rp < 0.2

# clean up stellar sample
use *= catalog.logg > 4.0                                 # surface gravity as proxy for main sequence
use *= (catalog.rcf - 1 < 0.05) + np.isnan(catalog.rcf)   # radius correction factor (Furlan+ 2017)
use *= catalog.ruwe < 1.4                                 # Gaia RUWE

# slice subpopulation
use *= ((catalog.npl >= MULTIPLICITY[0]) &
        (catalog.npl <= MULTIPLICITY[1]) &
        (catalog.period >= PER_LIM[0]) &
        (catalog.period <= PER_LIM[1]) &
        (catalog.mstar >= MSTAR_LIM[0]) &
        (catalog.mstar <= MSTAR_LIM[1]) &
        (catalog.rstar >= RSTAR_LIM[0]) &
        (catalog.rstar <= RSTAR_LIM[1]) &
        (catalog.feh >= FEH_LIM[0]) &
        (catalog.feh <= FEH_LIM[1]) &
        (catalog.teff >= TEFF_LIM[0]) &
        (catalog.teff <= TEFF_LIM[1]) &
        (catalog.age >= AGE_LIM[0]) &
        (catalog.age <= AGE_LIM[1])
       )

# update catalog
catalog = catalog.loc[use].reset_index(drop=True)
targets = np.array(catalog.planet_name)

In [ ]:
# Tophat binning
if RAD_LIM[0] != RAD_LIM[1]:
    use = (catalog[RAD_TYPE] >= RAD_LIM[0]) & (catalog[RAD_TYPE] <= RAD_LIM[1])
    
    catalog = catalog.loc[use].reset_index(drop=True)
    targets = np.array(catalog.planet_name)

    catalog['ln_binweight'] = np.zeros(len(targets))


# Gaussian binning
else:
    # grab objects within 3-sigma of bin center
    rad_center = RAD_LIM[0]
    rad_sigma  = np.sqrt(catalog[RAD_TYPE+'_err']**2 + (RAD_FWHM/2.355*rad_center)**2)
    rad_low    = np.max([0.1, np.min(rad_center-3*rad_sigma)])
    rad_high   = np.min([20., np.max(rad_center+3*rad_sigma)])

    use = np.abs(catalog[RAD_TYPE] - rad_center)/rad_sigma < 3.0
    
    catalog = catalog.loc[use].reset_index(drop=True)
    targets = np.array(catalog.planet_name)

    # numerically integrate probabilities for bins, accounting for measurement uncertainty
    Ngrid = 1000
    grid_over_range = np.linspace(np.log10(rad_low), np.log10(rad_high), Ngrid)
    weight_from_bin = stats.norm(np.log10(rad_center), RAD_FWHM/2.355*rad_center).pdf(grid_over_range)
    weight_from_obj = np.zeros((len(targets),Ngrid))

    for i, t in enumerate(targets):
        rad = catalog.loc[catalog.planet_name==t, RAD_TYPE]
        rad_err = catalog.loc[catalog.planet_name==t, RAD_TYPE+'_err']
        weight_from_obj[i] = stats.norm(np.log10(rad), rad_err/rad/np.log(10)).pdf(grid_over_range)

        do_plot=False
        if do_plot:
            plt.figure(figsize=(4,3))
            plt.plot(10**grid_over_range, weight_from_bin)
            plt.plot(10**grid_over_range, weight_from_obj[i])
            plt.show()

    catalog['ln_binweight'] = np.log(np.inner(weight_from_bin, weight_from_obj)) - 2*np.log(Ngrid)

#### Load posterior chains

In [ ]:
DR25_CHAINS = os.path.join(DATA_DIR, 'dr25-chains_trimmed-thinned.hdf')
chains  = {}
failure = []

# read in the data
for i, t in enumerate(targets):
    try:
        chains[t] = pd.DataFrame(load_dr25_data_from_hdf5(DR25_CHAINS, t))
        chains[t]['DUR14'] = calc_T14_circ(chains[t].PERIOD, chains[t].ROR, chains[t].IMPACT, chains[t].RHOTILDE)

        if np.any(chains[t].values < 0):
            raise ValueError("Negative values in posterior chain")
        if np.sum(np.isnan(chains[t].values)) > 0:
            raise ValueError("NaN values in posterior chain")
            
    except:
        warnings.warn("{0} failed to load".format(t))
        failure.append(t)

targets = list(np.array(targets)[~np.isin(targets,failure)])

print("{0} targets loaded".format(len(targets)))

#### Sanitize chains

In [ ]:
def gelman_rubin(c):
    J, L = c.shape
    
    mean_of_chain = np.mean(c, axis=1)
    mean_of_means = np.mean(mean_of_chain)

    B = L/(J-1) * np.sum((mean_of_chain - mean_of_means)**2)
    W = (1/J) * np.sum(1/(L-1) * np.sum((c.T - mean_of_chain)**2))

    Rhat = ((L-1)/L * W + B/L) / W

    return Rhat

In [ ]:
density = {}
failure = []

for i, t in enumerate(targets):
    # remove grazing transits
    if np.any(chains[t].IMPACT.values  > 1 - chains[t].ROR.values):
        failure.append(t)
    
    # eliminate NaN and zero-valued chains
    if np.any(chains[t].values < 0):
        failure.append(t)
    if np.sum(np.isnan(chains[t].values)) > 0:
        failure.append(t)

    # check Gelman-Rubin convergence statistic
    for k in chains[t].keys():
        Rhat = gelman_rubin(chains[t][k].values.reshape(4,-1))
        if Rhat > 1.05:
            failure.append(t)

    # check Hartigan dip test for multimodality
    for k in chains[t].keys():
        dip, pval = diptest.diptest(chains[t][k].values)
        if pval < 0.05:            
            failure.append(t)
    
    try:
        use = catalog.planet_name == t
        
        rho_mu = catalog.loc[use, 'rhostar'].iloc[0]
        rho_err1 = np.abs(catalog.loc[use, 'rhostar_err1'].iloc[0])
        rho_err2 = np.abs(catalog.loc[use, 'rhostar_err2'].iloc[0])

        # don't use highly asymmetric density constraints
        if np.abs(rho_err1-rho_err2)/(0.5*(rho_err1+rho_err2)) > 0.30:
            failure.append(t)
        else:
            density[t] = rho_mu, np.sqrt(rho_err1**2 + rho_err2**2)/np.sqrt(2)
    
    except:
        warnings.warn("{0} has no recorded density".format(t))
        failure.append(t)

targets = list(np.array(targets)[~np.isin(targets,failure)])
print("{0} targets found with unreliable chains, or posterior samples indicating b > 1 - Rp/Rs".format(len(np.unique(failure))))

## Importance sample $\{e,\omega,\rho_\star\}$

In [ ]:
print("Importance sampling...")

In [ ]:
from scipy.special import hyp2f1

def jacobian_integral(P, ror, b, T14):
    """
    P   : [days]
    T14 : [days]

    jac:
    """
    P_   = P*86400.       # [seconds]
    dur_ = T14*86400.     # [seconds]
    G    = apc.G.value    # Newton's constant

    const = (12*pi**3)/(P_**3*G) * (pi*dur_/P_)**-4
    dep = (1+ror)**3 * b * hyp2f1(-1.5,0.5,1.5, b**2/(1+ror)**2)

    return const*dep

In [ ]:
data = {}
failure = []

for i, t in enumerate(targets):
    try:
        # true stellar density (tuple) in g/cm3
        rho_true = 1.41*density[t][0], 1.41*density[t][1]
                   
        w, d = imp_sample_rhostar(chains[t], rho_true, ew_obs_prior=False, upsample=500)
        d = d.sample(n=NSAMP, replace=True, weights=w, ignore_index=True)

        J = jacobian(d.PERIOD, d.ROR, d.IMPACT, d.DUR14)
        d = d.sample(n=NSAMP, replace=True, weights=1/np.abs(J), ignore_index=True)
        
        data[t] = d

    except:
        warnings.warn("{0} failed during sampling and will not be included in the analysis".format(t))
        failure.append(t)

targets = list(np.array(targets)[~np.isin(targets,failure)])

# update catalog
catalog = catalog.loc[np.isin(catalog.planet_name, targets)].reset_index(drop=True)
targets = np.array(catalog.planet_name)

In [ ]:
DO_PLOT = True
if DO_PLOT:
    sns.set_context('paper', font_scale=1.2)
    
    if len(targets) > 50:
        targets_to_plot = np.random.choice(targets, size=50, replace=False)
    else:
        targets_to_plot = np.copy(targets)
    

    plt.figure(figsize=(5,5))
    for i, t in enumerate(targets_to_plot):
        e  = data[t].ECC.values
        e_ = np.linspace(0,1,1000)
    
        kde_e = stats.gaussian_kde(np.hstack([-e,e]), bw_method=0.1)
        pdf_e_ = 2*kde_e(e_)
    
        mode_e = e_[np.argmax(pdf_e_)]

        my_color = "lightgrey"
        my_cmap  = LinearSegmentedColormap.from_list("my_cmap", [my_color, "k"], N=50)
    
        plt.plot(e_, pdf_e_, c=my_cmap(mode_e), zorder=100*mode_e, lw=2)
        
    plt.xlim(0,1)
    plt.xlabel("$e$", fontsize=16)
    plt.ylabel("samples density", fontsize=16)
    plt.ylim(-0.1,5)
    plt.yticks([])
    
    #plt.text(0.98, 4.85, "Jovians", fontsize=16, color=my_color, ha='right', va='top')
    #plt.savefig(os.path.join(PROJECT_DIR, "Figures/ecc-posterior-kde-jovians.pdf"), bbox_inches='tight')
    #plt.text(0.98, 4.85, "$R_p$ = {0:.2f}-{1:.2f}".format(RAD_LIM[0],RAD_LIM[1]), fontsize=16, color=my_color, ha='right', va='top')
    #plt.savefig(os.path.join(PROJECT_DIR, "Figures/ecc-posterior-kde-{0:.2f}-{1:.2f}.pdf".format(RAD_LIM[0],RAD_LIM[1])), bbox_inches='tight')
    plt.show()

#### Vectorize quantities

In [ ]:
samples_array = {}

samples_array['ecc']    = np.zeros((len(targets),NSAMP))
samples_array['omega']  = np.zeros((len(targets),NSAMP))
samples_array['impact'] = np.zeros((len(targets),NSAMP))
samples_array['ln_wt']  = np.zeros((len(targets),NSAMP))

for i, t in enumerate(targets):
    samples_array['ecc'][i]    = np.array(data[t].ECC)
    samples_array['omega'][i]  = np.array(data[t].OMEGA)
    samples_array['impact'][i] = np.array(data[t].IMPACT)
    samples_array['ln_wt'][i]  = catalog.loc[catalog.planet_name == t, 'ln_binweight']

In [ ]:
N = len(targets)
ecc = np.median(samples_array['ecc'], axis=1)
use = ecc > np.percentile(ecc, 100*(N-8)/N)
high_e_catalog = catalog.loc[np.isin(catalog.planet_name, targets[use])]

high_e_catalog['planet_name period rp'.split()]

In [ ]:
#for i, t in enumerate(high_e_catalog.planet_name):
for i, t in enumerate(targets):
    print(t)

    fig, ax = plt.subplots(1, 3, figsize=(12,3))

    b_ = np.linspace(0,1,21)
    r_ = np.median(chains[t].ROR)
    T_ = np.median(chains[t].DUR14)
    P_ = np.median(chains[t].PERIOD)

    J_ = np.abs(jacobian(P_, r_, b_, T_))
    J_ = J_ / np.sum(J_) * 20
    
    ax[0].hist(chains[t].IMPACT, bins=np.linspace(0,1,21), color='lightgrey', density=True)
    ax[0].hist(chains[t].IMPACT, bins=np.linspace(0,1,21), color='k', histtype='step', density=True)
    ax[0].plot(b_, J_, color='r', lw=2)
    ax[0].set_title('Raw chains', fontsize=16)
    ax[0].set_yticks([])
    
    ax[1].hist(data[t].IMPACT, bins=np.linspace(0,1,21), color='lightgrey')
    ax[1].hist(data[t].IMPACT, bins=np.linspace(0,1,21), color='k', histtype='step')
    ax[1].set_title('Importance weighting', fontsize=16)
    ax[1].set_yticks([])

    w = stats.beta(a=0.52, b=3.76).pdf(data[t].ECC) * (1-data[t].ECC**2)
    w /= np.sum(w)

    ax[2].hist(data[t].IMPACT, weights=w, bins=np.linspace(0,1,21), color='lightgrey')
    ax[2].hist(data[t].IMPACT, weights=w, bins=np.linspace(0,1,21), color='k', histtype='step')
    ax[2].set_title('With shrinkage', fontsize=16)
    ax[2].set_yticks([])
    
    plt.show()

In [ ]:
#for i, t in enumerate(high_e_catalog.planet_name):
for i, t in enumerate(targets):
    print(t)

    w = stats.beta(a=0.52, b=3.76).pdf(data[t].ECC) * (1-data[t].ECC**2)
    w /= np.sum(w)
    
    fig, ax = plt.subplots(1, 3, figsize=(12,3))
    
    ax[0].hist(data[t].ECC, weights=None, bins=np.linspace(0,1,21), color='indigo', lw=2, histtype='step', density=True, label="raw data")
    ax[0].hist(data[t].ECC, weights=w, bins=np.linspace(0,1,21), color='mediumseagreen', lw=2, histtype='step', density=True, label="with shrinkage")
    ax[0].set_title('ECC', fontsize=20)
    ax[0].set_yticks([])
    ax[0].legend()
    
    ax[1].hist(data[t].OMEGA, weights=None, bins=21, color='indigo', lw=2, histtype='step', density=True)
    ax[1].hist(data[t].OMEGA, weights=w, bins=21, color='mediumseagreen', lw=2, histtype='step', density=True)
    ax[1].set_title('OMEGA', fontsize=20)
    ax[1].set_yticks([])

    ax[2].hist2d(data[t].ECC, data[t].OMEGA, weights=w, bins=(np.linspace(0,1,21), np.linspace(-0.5*pi,1.5*pi,21)))
    
    plt.show()

In [ ]:
for i, t in enumerate(targets):
    print(t)

    w = stats.beta(a=0.52, b=3.76).pdf(data[t].ECC) * (1-data[t].ECC**2)
    w /= np.sum(w)

    fig, ax = plt.subplots(1, 1, figsize=(3,3))

    bins = (np.linspace(0,1,51), np.linspace(data[t].DUR14.min(), data[t].DUR14.max(),51)*24)

    ax.hist2d(data[t].IMPACT, data[t].DUR14*24, weights=None, bins=bins)
    ax.set_xlabel("$b$")
    ax.set_ylabel("$T_{14}$ (hrs)")

    plt.savefig(os.path.join(PROJECT_DIR, "Figures/samples_histograms/{0}-b-T14.pdf".format(t)), bbox_inches='tight')
    plt.show()    

In [ ]:
#for i, t in enumerate(high_e_catalog.planet_name):
for i, t in enumerate(targets):
    print(t)

    w = stats.beta(a=0.52, b=3.76).pdf(data[t].ECC) * (1-data[t].ECC**2)
    w /= np.sum(w)
    
    fig, ax = plt.subplots(1, 2, figsize=(6,3))
    
    ax[0].hist2d(data[t].IMPACT, data[t].ECC, bins=(np.linspace(0,1,51), np.linspace(0,1,51)))
    ax[0].set_xlabel("$b$")
    ax[0].set_ylabel("$e$")

    ax[1].hist2d(data[t].OMEGA, data[t].ECC, bins=(np.linspace(-0.5*pi,1.5*pi,51), np.linspace(0,1,51)))
    ax[1].set_xlabel("$\omega$")
    ax[1].set_yticks([])

    plt.tight_layout()
    #plt.savefig(os.path.join(PROJECT_DIR, "Figures/samples_histograms/{0}-e-b-omega-raw.pdf".format(t)), bbox_inches='tight')
    plt.show()

In [ ]:
#for i, t in enumerate(high_e_catalog.planet_name):
for i, t in enumerate(targets):
    print(t)

    w = stats.beta(a=0.52, b=3.76).pdf(data[t].ECC) * (1-data[t].ECC**2)
    w /= np.sum(w)
    
    fig, ax = plt.subplots(1, 2, figsize=(6,3))
    
    ax[0].hist2d(data[t].IMPACT, data[t].ECC, weights=w, bins=(np.linspace(0,1,51), np.linspace(0,1,51)))
    ax[0].set_xlabel("$b$")
    ax[0].set_ylabel("$e$")

    ax[1].hist2d(data[t].OMEGA, data[t].ECC, weights=w, bins=(np.linspace(-0.5*pi,1.5*pi,51), np.linspace(0,1,51)))
    ax[1].set_xlabel("$\omega$")
    ax[1].set_yticks([])

    plt.tight_layout()
    plt.savefig(os.path.join(PROJECT_DIR, "Figures/samples_histograms/{0}-e-b-omega-shrunk.pdf".format(t)), bbox_inches='tight')
    plt.show()

## Run hierarchical model

In [ ]:
print("Running hierarchical MCMC using {0} planets".format(len(samples_array['ecc'])))

#### Load empirical distribution template (if needed)

In [ ]:
if DISTRIBUTION == 'empirical':
    template_values = np.loadtxt(os.path.join(PROJECT_DIR, "template_distribution.txt")).T
    template_spline = CubicSpline(template_values[0], template_values[1], extrapolate=False)
else:
    template_values = None
    template_spline = None

#### Build model and run MCMC

In [ ]:
# build model and sample from posterior
model, bin_edges = build_simple_model(samples_array,
                                      DISTRIBUTION,
                                      NBIN,
                                      e_detprior=E_DETPRIOR,
                                      b_detprior=B_DETPRIOR,
                                      template_spline=template_spline,
                                      eps=1e-6)

with model:
    trace = pmx.sample(tune=3000, draws=1000, chains=2, target_accept=0.95, return_inferencedata=True)

## Leave N Out validation

In [ ]:
npl = samples_array['ecc'].shape[0]
rank = np.array(stats.rankdata(np.median(samples_array['ecc'], axis=1)), dtype='int')
traces = []

for n in range(21):
    # remove N planets with the highest eccentricity
    use = rank <= npl - n
    
    samples_copy = {}
    for k in samples_array.keys():
        samples_copy[k] = samples_array[k][use]

    print("Running hierarchical MCMC using {0} planets".format(len(samples_copy['ecc'])))

    
    # build model and sample from posterior
    model, bin_edges = build_simple_model(samples_copy,
                                          DISTRIBUTION,
                                          NBIN,
                                          e_detprior=E_DETPRIOR,
                                          b_detprior=B_DETPRIOR,
                                          template_spline=template_spline,
                                          eps=1e-6)
    
    with model:
        trace = pmx.sample(tune=3000, draws=1000, chains=2, target_accept=0.95, return_inferencedata=True)

    # track results
    traces.append(trace)

In [ ]:
mean_ecc = np.zeros((len(traces),3))

for i, trace in enumerate(traces):
    mean_ecc[i] = np.percentile(trace.posterior.mean_ecc.values, [16,50,84], axis=(0,1))

x = np.arange(len(traces))
y = mean_ecc[:,1]
yerr = np.abs(mean_ecc[:,(0,2)].T - mean_ecc[:,1])

sns.set_context('paper', font_scale=1.2)

plt.figure(figsize=(5,5))
#plt.errorbar(x, y, yerr=yerr, fmt='ko')
plt.errorbar(x, y, yerr=yerr, fmt='o', color=my_color)
plt.xlim(-0.5,20.5)
plt.xticks([0,4,8,12,16,20])
plt.ylim(0,0.4)
plt.xlabel("number of planets removed", fontsize=16)
plt.ylabel("mean eccentricity", fontsize=16)
#plt.text(20, 0.39, "Jovians", fontsize=16, color=my_color, ha='right', va='top')
#plt.savefig(os.path.join(PROJECT_DIR, "Figures/leave-n-out-validation-jovians.pdf"), bbox_inches='tight')
#plt.text(20, 0.29, "$R_p$ = {0:.2f}-{1:.2f}".format(RAD_LIM[0],RAD_LIM[1]), fontsize=16, ha='right', va='top')
#plt.savefig(os.path.join(PROJECT_DIR, "Figures/leave-n-out-validation-{0:.2f}-{1:.2f}.pdf".format(RAD_LIM[0],RAD_LIM[1])), bbox_inches='tight')
plt.show()

## Save posterior trace

In [ ]:
# refactor dataframe for export
df = trace.to_dataframe(groups='posterior', include_coords=False)

column_map = {}
for k in list(df.keys()):    
    if k.find('[') == -1:
        column_map[k] = k
    else:
        column_map[k] = k[:k.find('[')] + '_{:02d}'.format(int(k[k.find('[')+1:-1]))

df = df.rename(columns=column_map)

In [ ]:
from astropy.io import fits

primary_hdu = fits.PrimaryHDU()
header = primary_hdu.header

header['YYYYMMDD'] = YYYYMMDD
header['DIST']     = DISTRIBUTION
header['NSAMP']    = NSAMP
header['NBIN']     = NBIN
header['NOBJ']     = samples_array['ecc'].shape[0]
header['MULT_0']   = MULTIPLICITY[0]
header['MULT_1']   = MULTIPLICITY[1]
header['PER_0']    = PER_LIM[0]
header['PER_1']    = PER_LIM[1]
header['RAD_TYPE'] = RAD_TYPE
header['RAD_0']    = RAD_LIM[0]
header['RAD_1']    = RAD_LIM[1]
header['RAD_FWHM'] = RAD_FWHM
header['MSTAR_0']  = MSTAR_LIM[0]
header['MSTAR_1']  = MSTAR_LIM[1]
header['RSTAR_0']  = RSTAR_LIM[0]
header['RSTAR_1']  = RSTAR_LIM[1]
header['FEH_0']    = FEH_LIM[0]
header['FEH_1']    = FEH_LIM[1]
header['TEFF_0']   = TEFF_LIM[0]
header['TEFF_1']   = TEFF_LIM[1]
header['AGE_0']    = AGE_LIM[0]
header['AGE_1']    = AGE_LIM[1]
header['E_PRIOR']  = E_DETPRIOR
header['B_PRIOR']  = B_DETPRIOR

samples_hdu = fits.BinTableHDU(data=df.to_records(index=False), name='SAMPLES')
binedges_hdu = fits.ImageHDU(bin_edges, name='BINEDGES')

hduL  = fits.HDUList([primary_hdu, samples_hdu, binedges_hdu])
fname = os.path.join(RESULTS_DIR, "{0}_{1}.fits".format(YYYYMMDD, RUN_ID))

hduL.writeto(fname, overwrite=True)

In [ ]:
# this complicated loading routine deals with big-endian vs little-endian mismatch between pandas and FITS
with fits.open(fname) as hduL:
    data = hduL['SAMPLES'].data
    keys = data.names
    
    _samples = []
    for k in keys:
        _samples.append(data[k])

    samples = pd.DataFrame(np.array(_samples).T, columns=keys)
    bin_edges = np.array(hduL['BINEDGES'].data)

## Inspect VanEylen+19 results

In [ ]:
# load VanEylen+19 catalog
VE19 = os.path.join(PROJECT_DIR, 'Catalogs/vaneylen_2019_asteroseismology_singles.txt')
ve19 = pd.read_csv(VE19, skiprows=4, skipfooter=1, delimiter='\t', usecols=[1,2,3], engine='python')
ve19 = ve19.rename(columns={'Unnamed: 1': 'koi_id', 'e (mode)': 'e_mode_ve', 'e (68%)': 'e_hdi_ve'})

koi_id_ve = []
e_mode_ve = []
e_hdi_ve_1 = []
e_hdi_ve_2 = []

for i, t in enumerate(ve19.koi_id):
    raw = ve19.e_hdi_ve[i][1:-1].split()
    
    koi_id_ve.append('K'+str(t[4:]).zfill(8))
    e_mode_ve.append(ve19.e_mode_ve[i])
    e_hdi_ve_1.append(float(raw[0].replace(',', '')))
    e_hdi_ve_2.append(float(raw[1].replace(',', '')))


ve19 = pd.DataFrame()
ve19['koi_id'] = koi_id_ve
ve19['e_mode_ve'] = e_mode_ve
ve19['e_hdi_ve_1'] = e_hdi_ve_1
ve19['e_hdi_ve_2'] = e_hdi_ve_2

In [ ]:
# load DR25 catalog
DR25_CATALOG = os.path.join(PROJECT_DIR, 'Catalogs/kepler_dr25_gaia_dr2_crossmatch.csv')
catalog = pd.read_csv(DR25_CATALOG, index_col=0)

keep = np.isin(catalog.planet_name, koi_id_ve)

catalog = catalog.loc[keep].reset_index(drop=True)
targets = np.array(catalog.planet_name)

In [ ]:
import arviz as az

koi_id_gp = []
e_mode_gp = []
e_hdi_gp_1 = []
e_hdi_gp_2 = []

for i, t in enumerate(targets):
    raw = az.hdi(data[t].ECC.values, hdi_prob=0.68)

    koi_id_gp.append(t)
    e_mode_gp.append(az.plots.plot_utils.calculate_point_estimate('mode', data[t].ECC.values).round(2))
    e_hdi_gp_1.append(raw[0].round(2))
    e_hdi_gp_2.append(raw[1].round(2))

gp24 = pd.DataFrame()
gp24['koi_id'] = koi_id_gp
gp24['e_mode_gp'] = e_mode_gp
gp24['e_hdi_gp_1'] = e_hdi_gp_1
gp24['e_hdi_gp_2'] = e_hdi_gp_2

In [ ]:
df = pd.concat([ve19.set_index('koi_id'), gp24.set_index('koi_id')], axis=1, join='inner').reset_index()
df.to_csv(os.path.join(PROJECT_DIR, "ve19_gp24_singles_compare.csv"))

In [ ]:
df.sort_values('e_mode_gp')

In [ ]:
x = df.e_mode_ve.values
y = df.e_mode_gp.values

xerr = np.abs((df.e_hdi_ve_1 - x, df.e_hdi_ve_2-x))
yerr = np.abs((df.e_hdi_gp_1 - y, df.e_hdi_gp_2-y))

plt.figure()
plt.errorbar(x, y, xerr=xerr, yerr=yerr, fmt='ko', alpha=0.3, capsize=3)
plt.plot(np.linspace(0,1), np.linspace(0,1), 'r:')
plt.xlabel("VE+19", fontsize=16)
plt.ylabel("GP24", fontsize=16)
plt.title("SINGLES", fontsize=20)
plt.show()

In [ ]:
plt.figure(figsize=(6,2))
plt.plot(x, y-x, 'ko', alpha=0.3)
plt.show()